In [2]:
from transformers import *
import os
import pandas as pd
import tokenizers

I0515 16:54:08.624496 140672851605248 file_utils.py:41] PyTorch version 1.4.0 available.
I0515 16:54:09.768602 140672851605248 file_utils.py:57] TensorFlow version 2.1.0 available.


In [17]:
# model_type = "roberta-base"

# tokenizer =  RobertaTokenizer(
#             vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
#             merges_file="transformers_vocab/{}-merges.txt".format(model_type),
#             lowercase=True,
#             add_prefix_space=True
#         )

model_type = "albert-base-v2"

tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
            lowercase=True,
        )

def get_prediction_v3(tokenizer, tweet, selected_text):
    
    tweet_copy = " ".join(str(tweet).split())
    tweet = " ".join(str(tweet).split()).replace("`", "'")
    selected_text = " ".join(str(selected_text).split()).replace("`", "'")
    
    tweet_copy = " " + tweet_copy.lower()
    tweet = " " + tweet.lower()
    selected_text = " " + selected_text.lower()
    
    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None
    
    # get char idx
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind + len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break
    
    # get char mask
    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    # get word offsets
    tweet_offsets = []
    tweet_offsets_token_level = []
    cursor = 0
    input_ids_orig = []
    
    for word in tweet.split():
        
        sub_words = tokenizer.tokenize(" " + word)
        encoded_word = tokenizer.convert_tokens_to_ids(sub_words)
#         print(" " + word, tokenizer.convert_ids_to_tokens(encoded_word))
        number_of_tokens = len(encoded_word)
        input_ids_orig += encoded_word
        
        start_offsets = cursor
        cursor += len(" " + word)
        end_offsets = cursor
        
        token_level_cursor = start_offsets
        
        for i in range(number_of_tokens):
            
            tweet_offsets.append((start_offsets, end_offsets))
            tweet_offsets_token_level.append((token_level_cursor, token_level_cursor + len(sub_words[i])))
            token_level_cursor += len(sub_words[i])
        
#     print(tweet_offsets)
#     print(tweet_offsets_token_level)
    
    # get word idx
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets_token_level):
#         print(tweet[offset1: offset2])
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]
    
#     print(targets_start, targets_end)
    
    
    return tweet_copy[tweet_offsets[targets_start][0]: tweet_offsets[targets_end][1]]

# tweet = "star wars ............ is **** boo??? i wanna do your job hand it over u can act as me at my high school lol"
# selected_text = "l"

tweet = "i`m soooooo sleeeeepy!!!"
selected_text = "soooooo sleeeeepy!!!"

get_prediction_v3(tokenizer, tweet, selected_text)

I0515 17:14:54.788259 140672851605248 tokenization_utils.py:420] Model name 'transformers_vocab/albert-base-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-base-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0515 17:14:54.789235 140672851605248 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0515 17:14:54.789624 140672851605248 tokenization_utils.py:502] loading file transformers_vocab/albert-base-v2-spiece.model


' soooooo sleeeeepy!!!'

In [ ]:
model_type = "roberta-base"

tokenizer_0 = RobertaTokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tokenizer_1 = tokenizers.ByteLevelBPETokenizer(
            vocab_file="transformers_vocab/{}-vocab.json".format(model_type),
            merges_file="transformers_vocab/{}-merges.txt".format(model_type),
            lowercase=True,
            add_prefix_space=True
        )


tweet = " SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cALLed LoSe f0LloWeRs FridAy... smH"


input_ids_orig_0 = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(tweet))

ids_0 = []
offsets_0 = []
offsets_0_token_level = []
cursor = 0


for i, word in enumerate(tweet.lower().split()):
    
    encoded_word = tokenizer_0.convert_tokens_to_ids(tokenizer_0.tokenize(" " + word))
    ids_0 += encoded_word
    
    number_of_tokens = len(encoded_word)

    start_offsets = cursor
    cursor += len(" " + word)
    end_offsets = cursor
    
    token_level_cursor = start_offsets

    for i in range(number_of_tokens):
        offsets_0.append((start_offsets, end_offsets))
        curr_token = tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(encoded_word[i]))
        
        offsets_0_token_level.append((token_level_cursor, token_level_cursor + len(curr_token)))
        token_level_cursor += len(curr_token)
    
# print(input_ids_orig_0)
# print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(input_ids_orig_0)))
print(ids_0)
print(offsets_0_token_level)
print(tokenizer_0.convert_tokens_to_string(tokenizer_0.convert_ids_to_tokens(ids_0)))

ids_1 = tokenizer_1.encode(tweet).ids
print(ids_1)
print(tokenizer_1.encode(tweet).offsets)
print(tokenizer_1.decode(ids_1))

In [7]:
train = pd.read_csv("/media/jionie/my_disk/Kaggle/Tweet/input/tweet-sentiment-extraction/train.csv")

In [18]:
model_type = "albert-base-v2"

tokenizer = AlbertTokenizer.from_pretrained(
            pretrained_model_name_or_path="transformers_vocab/{}-spiece.model".format(model_type),
            lowercase=True,
        )

for i in range(len(train)):
    
    label = str(train["selected_text"].iloc[i]).lower().strip()
    prediction = get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]).strip()
    
    if label != prediction:
        print(
              label, \
              "-------------------", \
              get_prediction_v3(tokenizer, train["text"].iloc[i], train["selected_text"].iloc[i]), \
              "-------------------", train["text"].iloc[i])

I0515 17:24:52.016807 140672851605248 tokenization_utils.py:420] Model name 'transformers_vocab/albert-base-v2-spiece.model' not found in model shortcut name list (albert-base-v1, albert-large-v1, albert-xlarge-v1, albert-xxlarge-v1, albert-base-v2, albert-large-v2, albert-xlarge-v2, albert-xxlarge-v2). Assuming 'transformers_vocab/albert-base-v2-spiece.model' is a path, a model identifier, or url to a directory containing tokenizer files.
W0515 17:24:52.017785 140672851605248 tokenization_utils.py:432] Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I0515 17:24:52.018180 140672851605248 tokenization_utils.py:502] loading file transformers_vocab/albert-base-v2-spiece.model


bullying me -------------------  bullying me... ------------------- my boss is bullying me...
onna -------------------  gonna ------------------- is back home now      gonna miss every one
oh marly, i`m so sorry!!  i hope you find her soon!! <3 <3 -------------------  oh marly, i`m so sorry!! i hope you find her soon!! <3 <3 -------------------  oh Marly, I`m so sorry!!  I hope you find her soon!! <3 <3
.no internet -------------------  malaysia...no internet ------------------- On the way to Malaysia...no internet access to Twit
well so much for being unhappy for about 10 minute -------------------  well so much for being unhappy for about 10 minutes. -------------------  If it is any consolation I got my BMI tested hahaha it says I am obesed  well so much for being unhappy for about 10 minutes.
ahhh, i slept through the game.  i`m gonna try my best to watch tomorrow though. i hope we play army. -------------------  ahhh, i slept through the game. i`m gonna try my best to watch tomorr

! i`m scared of him -------------------  role! i`m scared of him ------------------- Here`s a brief preview: http://bit.ly/eHCfP   OMG James is creepy in that role! I`m scared of him
sorr -------------------  sorry, -------------------    Sorry, we`ll try to keep it down.
i miss it there -------------------  i miss it there. -------------------  actually, I wish I was back in Tahoe.  I miss it there.
wack. -------------------  wack... -------------------  but my bday is JUNE 19.. this is wack... and ihavent seen any promotions for my bday party  someone better finagle this asap!
d there wasn`t a chance -------------------  and there wasn`t a chance -------------------  but it all went too quick and there wasn`t a chance lol
fail) -------------------  fail)' -------------------  'organization ...ITIL, forget about people (and so they fail)' preaching to the converted  IT is the people http://tr.im/kWUA
about to have dinner and then an evening of playing cards. already packed and ready t

.just not lucky, never have been. -------------------  now...just not lucky, never have been. -------------------  I`ve been unlocked for decades now...just not lucky, never have been.  Gottta make my own luck and that involves $$$$$
lame -------------------  night...lame ------------------- homework on a friday night...lame
: i admire you! you`re amazing! you inspire me to write  <3 -------------------  (: i admire you! you`re amazing! you inspire me to write <3 -------------------  I created my account just to get a chance to chat with you (: I ADMIRE YOU! you`re amazing! you inspire me to write  <3
i love being off work -------------------  again...i love being off work ------------------- Think I`ll go enjoy the sun`s rays again...I LOVE being off work
he`s not logging in yet.  and i doubt if he will check his his replies here in twitter. but we must keep on trying! -------------------  he`s not logging in yet. and i doubt if he will check his his replies here in twitter. but we mu

girsl night ! on my way to foam party  no sleep tonight !! -------------------  girsl night ! on my way to foam party no sleep tonight !! ------------------- Girsl night ! on my way to foam party  No sleep tonight !!
they grovelled   on tomorrow morning, will post a link to the show, should be on just after 9am if anyone is about -------------------  they grovelled on tomorrow morning, will post a link to the show, should be on just after 9am if anyone is about ------------------- They grovelled   On tomorrow morning, will post a link to the show, should be on just after 9am if anyone is about
i can`t believe i` -------------------  i can`t believe i`m ------------------- Sob! I can`t believe I`m ending my work week at the chapter about what Augusten Burrough`s father did to Ernie the guinea pig!
missed the ups guy again! ugh so sad  but i got some cool stuff today that i`m super excited about! and looking forward to tonight too -------------------  missed the ups guy again! ugh so sad

...

planning a road trip down to san diego- with a detour into vegas to celebrate my birthday! too bad it`s 7 wks away  i want to go now! -------------------  planning a road trip down to san diego- with a detour into vegas to celebrate my birthday! too bad it`s 7 wks away i want to go now! ------------------- Planning a road trip down to San Diego- With a detour into Vegas to celebrate my birthday! Too bad it`s 7 wks away  I want to go now!
hurt so badly?? -------------------  hurt so badly??!?!??!!? ------------------- UP is out today!!! Why why why why does my head hurt so badly??!?!??!!?   I want to see it!!!
yeah i guess so  it makes me very sad though becase the disc is scratched! -------------------  yeah i guess so it makes me very sad though becase the disc is scratched! -------------------  yeah I guess so  it makes me very sad though becase the disc is scratched!
got back and putting in the laundry. we got in there last  o well as long as its clean. #dsaa09 -------------------  

r fun -------------------  yr fun -------------------  excuse me ...theres me!!!  jkidding lol...so how long b4 yr fun night out starts lol
fair enough. actually, you dont have to give me x-men.  mad max will do me fine -------------------  fair enough. actually, you dont have to give me x-men. mad max will do me fine -------------------  fair enough. actually, you dont have to give me x-men.  mad max will do me fine
t. ew -------------------  it. eww ------------------- Monday...Funday!  Wake up people...and keep me awake please   Today is going to be a long one..I can already feel it. Eww
boring -------------------  homework...boring ------------------- homework...boring
e isn`t legal -------------------  marriage isn`t legal -------------------  **** marriage isn`t legal everywhere here.
im sorrryy  i went to the cinema with my broth.. cause im cool.. and yeah im sorrrry  tomorrow tho, ill make sure of -------------------  im sorrryy i went to the cinema with my broth.. cause im coo

all my worms either escaped or died  bums! -------------------  all my worms either escaped or died bums! ------------------- all my worms either escaped or died  bums!
is playing mybrute!  follow me at: http://yhana09.mybrute.com -------------------  is playing mybrute! follow me at: http://yhana09.mybrute.com ------------------- is playing MYBRUTE!  follow me at: http://yhana09.mybrute.com
at work  oh no it`s the first day of monthly visitor ahhhhhh -------------------  at work oh no it`s the first day of monthly visitor ahhhhhh ------------------- At work  oh no it`s the first day of monthly visitor ahhhhhh
o popular -------------------  also popular ------------------- _sydney is he on twitter?  I know  he is into photography. Runs also popular photography blog too.
i m so hating it -------------------  frend...i m so hating it ------------------- lif`s unfair....luk wat it did to ma frend...i m so hating it
i hate songs and surveys  they always seem to make me cry. ---------------

back to work tomoo  day 1 of 5 it will bee. -------------------  back to work tomoo day 1 of 5 it will bee. ------------------- back to work tomoo  day 1 of 5 it will bee.
i think the very late night on friday just hit me  off to town to meet a friend soon for ice cream and coffee -------------------  i think the very late night on friday just hit me off to town to meet a friend soon for ice cream and coffee -------------------  I think the very late night on Friday just hit me  off to town to meet a friend soon for ice cream and coffee
i don`t think i am going to get to see any of the sun today  gutted -------------------  i don`t think i am going to get to see any of the sun today gutted ------------------- I don`t think I am going to get to see any of the sun today  gutted I NEED COLOUR!
typical google docs is down  oh well open pages -------------------  typical google docs is down oh well open pages ------------------- typical Google Docs is down  oh well open Pages
i love  the na

? that seems totally out of line -------------------  'hoe'? that seems totally out of line -------------------  I`ve heard of disgruntled investors, but calling you a 'hoe'? That seems totally out of line
i would never do such a thing!  the only thing more awesomerer is the 15 minute walk to work -------------------  i would never do such a thing! the only thing more awesomerer is the 15 minute walk to work -------------------  I would never do such a thing!  The only thing more awesomerer is the 15 minute walk to work
staying in, hve june exams 4 docent training class ,need to study  , we hvng zombie movie sunday aftrn, u both shld come over! -------------------  staying in, hve june exams 4 docent training class ,need to study , we hvng zombie movie sunday aftrn, u both shld come over! -------------------  staying in, hve June exams 4 docent training class ,need to study  , we hvng zombie movie sunday aftrn, u both shld come over!
wanna leave work al -------------------  wanna leave

In [ ]:
tokenizer.tokenize(tweet)

In [ ]:
encoded = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet))
encoded

In [ ]:
decoded = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded))
decoded

In [ ]:
print(len(encoded), len(tokenizer.tokenize(tweet)))

In [ ]:
tweet.split()

In [ ]:
for (i, word) in enumerate(tweet.split()):
        
    sub_tokens = tokenizer.tokenize(word)

    print(sub_tokens)